<a href="https://colab.research.google.com/github/MichaelBillan/Cloud-Computing/blob/main/PROJECT_PANTHER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ================================================
# PART 1: INSTALL, IMPORTS & SETTINGS (OPTIMIZED)
# ================================================

# --------- 0) Fast, safe installs: install ONLY if missing ----------
import sys, importlib.util, subprocess

def pip_install_if_missing(packages):
    missing = []
    for pkg in packages:
        base = pkg.split("==")[0].split(">=")[0].split("[")[0]
        module = {
            "google-genai": "google.genai",
        }.get(base, base.replace("-", "_"))
        if importlib.util.find_spec(module) is None:
            missing.append(pkg)

    if missing:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", *missing])

# Base features (UI, Drive, Gemini, NLTK, PDF, Firebase, Big Data, Vision)
pip_install_if_missing([
    "gradio",
    "nltk",
    "fpdf",
    "google-genai",
    "google-api-python-client",
    "firebase-admin",
    "pyspark",
    "transformers==4.30.2"
])

# --------- 1) Imports (only what we actually use) ----------

import gradio as gr
import pandas as pd
import nltk
import re
import requests
import json
import matplotlib.pyplot as plt
import matplotlib.dates as mdates


from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from requests import get as http_get
from os import environ
from bs4 import BeautifulSoup
from collections import defaultdict
from urllib.parse import urlparse

from google import genai

from fpdf import FPDF
from PIL import Image as PILImage
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime
import pytz

import firebase_admin
from firebase_admin import credentials, db as firebase_db

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, min as spark_min, max as spark_max, avg as spark_avg, stddev as spark_stddev

# Note: Importing the library is fast, we still load the MODEL lazily in Part 2.
from transformers import AutoImageProcessor, AutoModelForImageClassification
import torch

GEMINI_API_KEY = "" # generated by Michael at 09/02

BASE_URL = "https://server-cloud-v645.onrender.com/"
MODEL_ID = "wambugu71/crop_leaf_diseases_vit"


# --------- 3) NLTK downloads only if missing ----------
def nltk_download_if_missing(resource_path, download_name=None):
    try:
        nltk.data.find(resource_path)
    except LookupError:
        nltk.download(download_name or resource_path.split("/")[-1], quiet=True)

nltk_download_if_missing("corpora/stopwords", "stopwords")
nltk_download_if_missing("tokenizers/punkt", "punkt")

# --------- 4) Globals ----------
GLOBAL_CACHE = {"temperature": None, "humidity": None, "soil": None}
DOC_TITLES = {}
ARTICLES = {}
drive_service = None
engine = None
GENAI_CLIENT = None
plant_classifier = None
FIREBASE_APP = None
FIREBASE_DB_URL = None
SPARK = None

In [ ]:
# ==========================================
# PART 2: HELPER FUNCTIONS & CLASSES
# ==========================================

IL_TZ = pytz.timezone('Asia/Jerusalem') # Time zone configuration

# ---------------------------
# 1) WEB FETCH + MAIN TEXT EXTRACTOR
# ---------------------------
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    )
}

def fetch_html(url: str, timeout: int = 10) -> str:
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    return r.text

def _clean_text(text: str) -> str:
    text = re.sub(r"\s+", " ", text).strip()
    return text

def _remove_noise(soup: BeautifulSoup) -> None:
    for tag in soup(["script", "style", "noscript", "header", "footer", "nav", "aside"]):
        tag.decompose()

def extract_main_article_text(url: str, html: str) -> dict:

    soup = BeautifulSoup(html, "html.parser")
    _remove_noise(soup)

    title = None
    if soup.title and soup.title.get_text(strip=True):
        title = soup.title.get_text(strip=True)

    h1 = soup.find("h1")
    if h1 and h1.get_text(strip=True):
        title = h1.get_text(strip=True)

    selectors = [
      "div.main-content",
      "div.c-article-body",
      "article",
      "main",
      "div[itemprop='articleBody']",
      "section[aria-labelledby='abstract']",
      "div#content",
      "div#main-content",
    ]

    chosen = None
    for sel in selectors:
        node = soup.select_one(sel)
        if node:
            txt = node.get_text(" ", strip=True)
            if txt and len(txt) > 600:
                chosen = txt
                break

    # --- Fallback: pick the largest reasonable text block ---
    if not chosen:
        candidates = []
        for node in soup.find_all(["article", "main", "section", "div"]):
            txt = node.get_text(" ", strip=True)
            # keep only blocks that look like real content
            if txt and len(txt) > 800:
                candidates.append((len(txt), txt))
        if candidates:
            candidates.sort(key=lambda x: x[0], reverse=True)
            chosen = candidates[0][1]

    # Final fallback: whole page text (worst case)
    if not chosen:
        chosen = soup.get_text(" ", strip=True)

    return {
        "title": title or url,
        "content": _clean_text(chosen),
        "source": url
    }

# ---------------------------
# 2) BUILD YOUR ARTICLES/TITLES FROM URLS
# ---------------------------
def load_articles_from_web(urls: list[str]) -> tuple[dict, dict]:

    for i, url in enumerate(urls, 1):
        doc_id = str(i)
        try:
            html = fetch_html(url)
            extracted = extract_main_article_text(url, html)

            DOC_TITLES[doc_id] = extracted["title"]
            ARTICLES[doc_id] = extracted["content"]

            print(f"✅ Loaded [{doc_id}] {extracted['title']}")
        except Exception as e:
            print(f"❌ Failed loading {url}: {e}")

    return ARTICLES, DOC_TITLES


# --- 2. Search Engine Logic (Class) ---
class LectureSearchEngine:
    def __init__(self):
        self.word_locations = defaultdict(list)
        self.documents = {}
        self.titles = {}
        self.stemmer = PorterStemmer()
        self.stop_words = set(stopwords.words('english'))
        self.stop_words.update({'a', 'an', 'the', 'and', 'or', 'in', 'on', 'at', 'to', 'for', 'of', 'with'})

    def stem_word(self, word):
        return self.stemmer.stem(word.lower())

    def build_index(self, docs, titles):
        self.documents = docs
        self.titles = titles
        self.word_locations.clear()

        for doc_id, content in self.documents.items():
            raw_words = re.findall(r'\w+', content.lower())
            word_counts = defaultdict(int)

            for word in raw_words:
                if word not in self.stop_words:
                    word_counts[self.stem_word(word)] += 1

            for word, count in word_counts.items():
                self.word_locations[word].append((doc_id, count))

    def get_context(self, content, query_words, window=180):
        content_lower = content.lower()
        best_idx = -1

        for word in query_words:
            root = word[:4] if len(word) > 4 else word
            idx = content_lower.find(root)

            if idx != -1:
                best_idx = idx
                break

        if best_idx != -1:
            start = max(0, best_idx - 60)
            end = min(len(content), best_idx + window)
            return "..." + content[start:end].replace("\n", " ") + "..."
        return content[:220].replace("\n", " ") + "..."

    def search(self, query, num_results=3):
        raw_query_words = [w.lower() for w in re.findall(r'\w+', query) if w.lower() not in self.stop_words]
        stemmed_query_words = [self.stem_word(w) for w in raw_query_words]

        if not stemmed_query_words:
            return []

        page_scores = defaultdict(lambda: {'matches': 0, 'total_freq': 0})

        for word in stemmed_query_words:
            for doc_id, freq in self.word_locations.get(word, []):
                page_scores[doc_id]['matches'] += 1
                page_scores[doc_id]['total_freq'] += freq

        ranked = [(doc_id, s['matches'], s['total_freq']) for doc_id, s in page_scores.items()]
        ranked.sort(key=lambda x: (x[1], x[2]), reverse=True)

        results = []
        for doc_id, matches, total_freq in ranked[:num_results]:
            title = self.titles.get(doc_id, "Unknown")
            content = self.documents.get(doc_id, "")
            context = self.get_context(content, raw_query_words)
            results.append({
                'title': title,
                'score': f"Matches: {matches}, Freq: {total_freq}",
                'context': context
            })
        return results


# --- 3. Gemini & RAG Helpers ---
def get_genai_client():
    global GENAI_CLIENT
    if GENAI_CLIENT is None:
        GENAI_CLIENT = genai.Client(api_key=GEMINI_API_KEY)
    return GENAI_CLIENT

def get_working_model():
    return "gemini-2.5-flash"

def search_engine_rag(query):
    if not ARTICLES:
        return "⚠️ Error: No documents loaded."
    results = engine.search(query)
    if not results:
        return f"No results found for: '{query}'"

    output_log = f"🔎 Found {len(results)} docs (Ranked by Matches & Freq)\n" + "=" * 40 + "\n"
    context_text = []

    for res in results:
        output_log += f"\n📄 [{res['title']}] ({res['score']})\n - {res['context']}\n"
        context_text.append(f"Source ({res['title']}): {res['context']}")

    try:
        client = get_genai_client()
        prompt = (
            f"Question: {query}\n"
            f"Base your answer ONLY on the following context:\n" + "\n".join(context_text)
        )

        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=prompt
        )
        gemini_summary = f"\n🤖 AI Answer:\n{response.text}\n"
    except Exception as e:
        gemini_summary = f"\n(AI Error: {e})\n"


    return gemini_summary + "\n" + output_log

def get_index_table():
    data = []
    if engine:
        for i, (word, locs) in enumerate(engine.word_locations.items()):
            if i >= 100:
                break
            data.append({"term": word, "docs": str(locs)})
    return pd.DataFrame(data)


# --- 4. IoT Helpers ---
def fetch_data_as_df(feed, limit, save_to_firebase=True):
    try:
        resp = http_get(f"{BASE_URL}/history", params={"feed": feed, "limit": limit}, timeout=3)
        data = resp.json()
        if "data" in data and len(data["data"]) > 0:
            df = pd.DataFrame(data["data"])

            # --- TIMEZONE FIX START ---
            df["created_at"] = pd.to_datetime(df["created_at"])
            if df["created_at"].dt.tz is None:
                df["created_at"] = df["created_at"].dt.tz_localize('UTC')
            df["created_at"] = df["created_at"].dt.tz_convert(IL_TZ)
            df["created_at"] = df["created_at"].dt.tz_localize(None)
            # --- TIMEZONE FIX END ---

            df["value"] = pd.to_numeric(df["value"], errors="coerce")
            df = df.sort_values("created_at")
            GLOBAL_CACHE[feed] = df

            if save_to_firebase:
              try:
                msg = save_iot_df_to_firebase(feed, df)
                print(msg)
              except Exception as e:
                print(f"Error saving to Firebase: {e}")

            return df
    except:
        return None
    return None


def create_plot(df, title, color):
    if df is None or df.empty:
        return None
    fig, ax = plt.subplots(figsize=(8, 3.5))
    ax.plot(df["created_at"], df["value"], marker='.', linestyle='-', color=color, linewidth=1.5)
    ax.set_title(title)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    return fig

LEADERBOARD_DATA = [
    {"Player": "Foaad", "Score": 255},
    {"Player": "You 🫵", "Score": 245},
    {"Player": "Michael", "Score": 90},
    {"Player": "Yazan", "Score": 75},
    {"Player": "Baraah", "Score": 60},
    {"Player": "Rami", "Score": 15}
]

def get_leaderboard_df():
    df = pd.DataFrame(LEADERBOARD_DATA)
    df = df.sort_values(by="Score", ascending=False).reset_index(drop=True)
    df.insert(0, 'Rank', range(1, 1 + len(df)))
    return df

def update_leaderboard():
    global LEADERBOARD_DATA
    for player in LEADERBOARD_DATA:
        if "You" in player["Player"]:
            player["Score"] += 5
            break
    return get_leaderboard_df()

# --- OPTIMIZED PARALLEL FETCH (LIMIT=1) ---
def fetch_sensor_values(ignored_limit):
    """
    Fetches ONLY the latest value (limit=1) for all sensors in PARALLEL.
    """
    val_temp, val_hum, val_soil = "Loading...", "Loading...", "Loading..."
    REAL_TIME_LIMIT = 1

    with ThreadPoolExecutor(max_workers=3) as executor:
        future_t = executor.submit(fetch_data_as_df, "temperature", REAL_TIME_LIMIT)
        future_h = executor.submit(fetch_data_as_df, "humidity", REAL_TIME_LIMIT)
        future_s = executor.submit(fetch_data_as_df, "soil", REAL_TIME_LIMIT)

        df_t = future_t.result()
        df_h = future_h.result()
        df_s = future_s.result()

    if df_t is not None and not df_t.empty:
        val_temp = f"{df_t.iloc[-1]['value']} °C"
    else:
        val_temp = "---"

    if df_h is not None and not df_h.empty:
        val_hum = f"{df_h.iloc[-1]['value']} %"
    else:
        val_hum = "---"

    if df_s is not None and not df_s.empty:
        val_soil = str(df_s.iloc[-1]['value'])
    else:
        val_soil = "---"

    current_time = datetime.now(IL_TZ).strftime("%H:%M:%S")
    update_msg = f"⏱️ Last Updated (IL): {current_time}"

    return val_temp, val_hum, val_soil, update_msg

def export_json_data(limit):
    export_data = {}
    feeds = ["temperature", "humidity", "soil"]

    for feed in feeds:
        df = fetch_data_as_df(feed, limit)
        if df is not None and not df.empty:
            export_data[feed] = df.to_dict(orient="records")

    json_filename = "iot_data.json"
    try:
        with open(json_filename, 'w') as f:
            json.dump(export_data, f, default=str)
        return gr.File(value=json_filename, visible=True)
    except Exception as e:
        return gr.File(visible=False)

def refresh_dashboard_real():
    limit = 20
    df_t = fetch_data_as_df("temperature", limit)
    df_h = fetch_data_as_df("humidity", limit)
    df_s = fetch_data_as_df("soil", limit)

    fig_t = create_plot(df_t, "Temp Trend", "#ff6b6b")
    fig_h = create_plot(df_h, "Hum Trend", "#4ecdc4")
    fig_s = create_plot(df_s, "Soil Trend", "#8d6e63")

    val_t = df_t.iloc[-1]["value"] if df_t is not None and not df_t.empty else 0
    val_h = df_h.iloc[-1]["value"] if df_h is not None and not df_h.empty else 0
    val_s = df_s.iloc[-1]["value"] if df_s is not None and not df_s.empty else 0

    status = "Warning ⚠️" if val_t > 35 else "OK ✅"
    return fig_t, fig_h, fig_s, f"{val_t} °C", f"{val_h} %", f"{val_s}", status



# --- 5. Image AI Helpers (LAZY LOAD MODEL) ---
# --- HF Model (Clean, Direct Load) ---

def normalize_disease_label(label: str) -> str:
    """
    Convert model labels into a clean phrase for Gemini:
    e.g. 'Tomato___Late_blight' -> 'Tomato Late blight'
         'Potato___healthy' -> 'Potato healthy'
         'Tomato___Leaf_Mold' -> 'Tomato Leaf Mold'
    """
    if not label:
        return "unknown disease"

    # common PlantVillage-style formatting
    s = label.replace("___", " ").replace("__", " ").replace("_", " ")
    s = re.sub(r"\s+", " ", s).strip()

    # normalize some common tokens
    s = s.replace("Leaf Mold", "Leaf mold")
    s = s.replace("Late blight", "late blight")
    s = s.replace("Early blight", "early blight")

    return s


processor = AutoImageProcessor.from_pretrained(MODEL_ID)
model = AutoModelForImageClassification.from_pretrained(MODEL_ID)

def get_plant_model():
    plant_processor = None
    plant_model = None
    if plant_processor is None:
        plant_processor = AutoImageProcessor.from_pretrained(MODEL_ID)
    if plant_model is None:
        plant_model = AutoModelForImageClassification.from_pretrained(MODEL_ID)
        plant_model.eval()
    return plant_processor, plant_model

def analyze_image(img):
    if img is None:
        return "⚠️ Please upload an image."

    try:
        processor, model = get_plant_model()

        # numpy -> PIL
        pil_img = PILImage.fromarray(img.astype("uint8")).convert("RGB")

        # preprocess
        inputs = processor(images=pil_img, return_tensors="pt")

        # inference
        with torch.no_grad():
            outputs = model(**inputs)
            probs = torch.softmax(outputs.logits, dim=-1)[0]
            top_idx = int(torch.argmax(probs).item())
            score = float(probs[top_idx].item())

        raw_label = model.config.id2label.get(top_idx, str(top_idx))
        nice_label = normalize_disease_label(raw_label)

        # Decide "healthy" robustly (covers: healthy, Tomato healthy, Potato healthy, etc.)
        is_healthy = "healthy" in raw_label.lower() or raw_label.lower().endswith(" healthy")

        if is_healthy:
            diagnosis = f"✅ Healthy ({nice_label})\nConfidence: {score:.1%}"
            advice = gemini_care_advice("healthy plant / general plant care", score)
            return diagnosis + "\n\n🛠️ Care / Prevention:\n" + advice

        diagnosis = f"⚠️ Potential Issue: {nice_label}\nConfidence: {score:.1%}"

        # Pass a clean disease string (better Gemini recommendations)
        advice = gemini_care_advice(nice_label, score)

        return diagnosis + "\n\n🛠️ How to handle it:\n" + advice

    except Exception as e:
        return f"❌ Error: {str(e)}"



# --- 6. PDF Report Helpers ---
class PDFReport(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 15)
        self.cell(0, 10, 'IoT Big Data Analytics Report', 0, 1, 'C')
        self.ln(5)

    def chapter_title(self, title):
        self.set_font('Arial', 'B', 12)
        self.set_fill_color(200, 220, 255)
        self.cell(0, 10, title, 0, 1, 'L', 1)
        self.ln(4)

    # Function to display Analytics Table in PDF
    def chapter_analytics_table(self, df):
        self.set_font('Arial', 'B', 10)
        # Table Headers
        self.cell(40, 8, 'Feed', 1)
        self.cell(30, 8, 'Min', 1)
        self.cell(30, 8, 'Max', 1)
        self.cell(30, 8, 'Avg', 1)
        self.cell(40, 8, 'Anomalies (3s)', 1)
        self.ln()

        self.set_font('Arial', '', 10)
        if df is not None and not df.empty:
            for _, row in df.iterrows():
                feed_name = str(row.get('feed', 'N/A'))
                val_min = f"{float(row.get('min', 0)):.2f}"
                val_max = f"{float(row.get('max', 0)):.2f}"
                val_avg = f"{float(row.get('avg', 0)):.2f}"
                val_anom = str(row.get('anomalyCount_3sigma', 0))

                self.cell(40, 8, feed_name, 1)
                self.cell(30, 8, val_min, 1)
                self.cell(30, 8, val_max, 1)
                self.cell(30, 8, val_avg, 1)
                self.cell(40, 8, val_anom, 1)
                self.ln()
        else:
             self.cell(0, 8, "No Analytics Data Available from Firebase", 1, 1)
        self.ln(10)

    def chapter_body_raw(self, df):
        self.set_font('Arial', '', 10)
        self.cell(90, 8, 'Timestamp', 1)
        self.cell(40, 8, 'Value', 1)
        self.ln()
        if df is not None and not df.empty:
            for _, row in df.sort_values("created_at", ascending=False).head(20).iterrows():
                self.cell(90, 8, str(row['created_at']), 1)
                self.cell(40, 8, str(row['value']), 1)
                self.ln()
        self.ln(10)

# PDF Generation Function
def generate_pdf_report():
    # Fetch Analytics directly from Firebase
    df_analytics = load_latest_analytics_from_firebase()

    pdf = PDFReport()
    pdf.add_page()

    # Section 1: Big Data Overview
    pdf.chapter_title("Big Data Analytics Overview (from Firebase)")
    pdf.chapter_analytics_table(df_analytics)

    # Section 2: Raw Snippets
    pdf.chapter_title("Recent Raw Data Snippets")
    for feed in ["temperature", "humidity", "soil"]:
        # Try to get from cache or fetch small amount
        df = GLOBAL_CACHE.get(feed)
        if df is None:
             df = fetch_data_as_df(feed, 10, save_to_firebase=False)

        pdf.set_font('Arial', 'B', 10)
        pdf.cell(0, 10, f"Feed: {feed}", 0, 1)
        pdf.chapter_body_raw(df)

    filename = "iot_bigdata_report.pdf"
    pdf.output(filename)
    return filename, f"✅ Professional Report (Firebase Data) Saved to {filename}"


# --- 7. Firebase Realtime DB Helpers ---

FIREBASE_SERVICE_ACCOUNT = {
}



def firebase_init(db_url: str):
    global FIREBASE_APP, FIREBASE_DB_URL

    FIREBASE_DB_URL = db_url

    if not firebase_admin._apps:
        cred = credentials.Certificate(FIREBASE_SERVICE_ACCOUNT)
        FIREBASE_APP = firebase_admin.initialize_app(
            cred,
            {"databaseURL": db_url}
        )
    else:
        FIREBASE_APP = firebase_admin.get_app()

    return True


def fb_set(path: str, value):
    ref = firebase_db.reference(path)
    ref.set(value)

def fb_push(path: str, value):
    ref = firebase_db.reference(path)
    ref.push(value)

def fb_get(path: str):
    ref = firebase_db.reference(path)
    return ref.get()

def save_index_to_firebase(engine: LectureSearchEngine):
    if engine is None or not engine.documents:
        return "⚠️ No index to save."

    index_payload = {}

    for term, locs in engine.word_locations.items():
        index_payload[term] = {
            str(doc_id): int(freq)
            for doc_id, freq in locs
        }

    meta = {
        "createdAt": datetime.now(pytz.UTC).isoformat(),
        "docCount": len(engine.documents),
        "termCount": len(engine.word_locations),
    }

    fb_set("/indexes/lecture_search/latest/meta", meta)
    fb_set("/indexes/lecture_search/latest/terms", index_payload)

    return f"✅ Index saved correctly. terms={meta['termCount']}"

# Load index FROM Firebase

def load_index_from_firebase(engine):
    print("⏳ Checking Firebase for existing search index...")
    try:
        terms_data = fb_get("/indexes/lecture_search/latest/terms")

        if not isinstance(terms_data, dict) or not terms_data:
            print("⚠️ No valid index found in Firebase.")
            return False

        engine.word_locations.clear()

        loaded_terms = 0
        loaded_pairs = 0

        for term, doc_map in terms_data.items():
            if not isinstance(doc_map, dict) or not doc_map:
                continue

            # ✅ Normalize term so it matches search() lookup
            norm_term = engine.stem_word(str(term).strip().lower())

            locs = []
            for d_id, freq in doc_map.items():
                try:
                    locs.append((str(d_id), int(freq)))
                except:
                    continue

            if not locs:
                continue

            # Merge (in case multiple raw keys normalize to same stem)
            engine.word_locations[norm_term].extend(locs)
            loaded_terms += 1
            loaded_pairs += len(locs)

        # ✅ Sanity checks (prevents "loads fine but searches always empty")
        if loaded_terms == 0:
            print("⚠️ Loaded 0 usable terms (bad structure).")
            return False

        # Strong check: ensure at least one common term exists
        common = engine.stem_word("plant")
        if common not in engine.word_locations:
            print("⚠️ Loaded index but it doesn't contain common term 'plant' -> treating as invalid.")
            return False

        print(f"✅ Loaded {loaded_terms} terms ({loaded_pairs} postings) from Firebase!")
        return True

    except Exception as e:
        print(f"❌ Error loading index from Firebase: {e}")
        return False



def save_iot_df_to_firebase(feed: str, df: pd.DataFrame, max_rows: int = 2000):

    if df is None or df.empty:
        return f"⚠️ No data for feed={feed}"

    dfx = df.copy()

    dfx["created_at"] = pd.to_datetime(dfx["created_at"], errors="coerce")
    dfx = dfx.dropna(subset=["created_at"])

    dfx["ts_ms"] = (dfx["created_at"].astype("int64") // 1_000_000).astype("int64")

    dfx = dfx.tail(max_rows)

    # Read last saved timestamp
    meta_path = f"/iot/raw_meta/{feed}/last_ts_ms"
    last_ts = fb_get(meta_path)

    try:
        last_ts = int(last_ts) if last_ts is not None else 0
    except:
        last_ts = 0

    # Filter only new rows
    new_rows = dfx[dfx["ts_ms"] > last_ts].copy()
    if new_rows.empty:
        return f"✅ No new rows to save for {feed} (dedup OK)"

    base_path = f"/iot/raw/{feed}"

    # Write each row by deterministic key (set overwrites same key, so no duplicates)
    max_written_ts = last_ts
    for _, row in new_rows.iterrows():
        ts = int(row["ts_ms"])
        payload = {
            "created_at": str(row["created_at"]),
            "value": None if pd.isna(row.get("value")) else float(row["value"]) if str(row.get("value")).replace('.','',1).isdigit() else row.get("value")
        }
        fb_set(f"{base_path}/{ts}", payload)
        if ts > max_written_ts:
            max_written_ts = ts

    # Update meta with latest written timestamp
    fb_set(meta_path, int(max_written_ts))

    return f"✅ Saved {len(new_rows)} NEW rows (dedup) to {base_path}"


# --- 8. Big Data (Spark) Analytics on IoT ---

def spark_get_session():
    global SPARK
    if SPARK is not None:
        return SPARK
    SPARK = SparkSession.builder.appName("IoT Big Data Analytics").getOrCreate()
    return SPARK

def compute_bigdata_analytics_from_df(feed: str, df: pd.DataFrame):
    """
    Big data analytics for one feed:
    - Required: min/max (per tutorial)
    - Extra: avg/std + anomaly count (z-score style)
    Saves results to Firebase under /iot/analytics/latest/{feed}
    """
    if df is None or df.empty:
        return None, f"⚠️ No data for {feed}"

    # Prepare clean pandas
    dfx = df[["created_at", "value"]].dropna().copy()
    dfx["created_at"] = pd.to_datetime(dfx["created_at"])
    dfx["value"] = pd.to_numeric(dfx["value"], errors="coerce").dropna()

    if dfx.empty:
        return None, f"⚠️ No numeric data for {feed}"

    spark = spark_get_session()

    # Spark DF
    sdf = spark.createDataFrame(dfx.assign(created_at=dfx["created_at"].astype(str)).to_dict(orient="records"))

    # Spark aggregations
    agg_row = sdf.agg(
        spark_min(col("value")).alias("min"),
        spark_max(col("value")).alias("max"),
        spark_avg(col("value")).alias("avg"),
        spark_stddev(col("value")).alias("stddev"),
    ).collect()[0]

    stats = {
        "feed": feed,
        "count": int(sdf.count()),
        "min": float(agg_row["min"]) if agg_row["min"] is not None else None,
        "max": float(agg_row["max"]) if agg_row["max"] is not None else None,
        "avg": float(agg_row["avg"]) if agg_row["avg"] is not None else None,
        "stddev": float(agg_row["stddev"]) if agg_row["stddev"] is not None else None,
    }

    # MapReduce-style min/max using RDD (conceptual match)
    rdd = sdf.select("value").rdd.map(lambda row: float(row["value"]))
    mr_min = rdd.reduce(lambda a, b: a if a < b else b)
    mr_max = rdd.reduce(lambda a, b: a if a > b else b)
    stats["mapreduce_min"] = float(mr_min)
    stats["mapreduce_max"] = float(mr_max)

    # Simple anomaly count using z-score (if stddev exists and > 0)
    if stats["stddev"] and stats["stddev"] > 0:
        mean = stats["avg"]
        sd = stats["stddev"]
        anomalies = dfx[(dfx["value"] - mean).abs() > 3 * sd]
        stats["anomalyCount_3sigma"] = int(len(anomalies))
    else:
        stats["anomalyCount_3sigma"] = 0

    stats["generatedAt"] = datetime.now(pytz.UTC).isoformat()

    # Save to Firebase
    fb_set(f"/iot/analytics/latest/{feed}", stats)

    return stats, f"✅ Big data analytics computed & saved for {feed}"

def run_bigdata_pipeline(chk_temp, chk_hum, chk_soil, limit):
    feeds = []
    if chk_temp: feeds.append("temperature")
    if chk_hum: feeds.append("humidity")
    if chk_soil: feeds.append("soil")

    if not feeds:
        return pd.DataFrame([]), None, "⚠️ Choose at least one feed."

    all_stats = []
    last_fig = None
    logs = []

    for feed in feeds:
        df = fetch_data_as_df(feed, int(limit), save_to_firebase=True)
        stats, msg = compute_bigdata_analytics_from_df(feed, df)
        logs.append(msg)

        if stats:
            all_stats.append(stats)

        # Plot last selected feed
        if df is not None and not df.empty:
            last_fig = create_plot(df, f"{feed} Trend (for analytics)", "green")

    return pd.DataFrame(all_stats), last_fig, "\n".join(logs)

def load_latest_analytics_from_firebase():
    data = fb_get("/iot/analytics/latest")
    if not data:
        return pd.DataFrame([])

    # data is {feed: stats}
    rows = []
    for feed, stats in data.items():
        if isinstance(stats, dict):
            rows.append(stats)
    return pd.DataFrame(rows)


# ==========================================
# CHATBOT LOGIC (GenAI) - FIXED to 2.5 Flash
# ==========================================

def gemini_care_advice(disease_label: str, confidence: float | None = None) -> str:
    """
    Returns 2–3 short, safe care steps for the detected disease.
    """
    try:
        client = get_genai_client()
        conf_txt = f"{confidence:.1%}" if confidence is not None else "unknown"

        prompt = f"""
You are an agronomy assistant.
Disease detection result: "{disease_label}" (confidence: {conf_txt}).

Give:
1) 2–3 short handling steps (bullet points).
2) One prevention tip.
Keep it practical and non-technical.
If the label is unclear, ask the user what plant it is.
"""

        resp = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=prompt.strip()
        )
        return resp.text.strip()
    except Exception as e:
        return f"(Advice Error: {e})"


def chat_with_gemini(message, history):
    """
    Chatbot function for Tab 7 (stateless).
    Uses the new google-genai Client API.
    """
    try:
        client = get_genai_client()

        system_instruction = (
            "You are a helpful AI assistant for a Smart Plant IoT System. "
            "You help users understand plant health, sensor data (Temperature, Humidity, Soil Moisture), "
            "and big data analytics. "
            "Be concise, friendly, and professional."
            "system architecture: 1.Image upload for plant disease detection. 2.IoT Data. 3.Big Data Observations. 4.Search Engine Articles. 5.Dashboard with IoT data graphs. 6.Generate PDF report. 7.AI Chatbot."
        )

        # Optional: include last few turns from Gradio history to improve continuity
        # history is typically list[tuple(user, bot)]
        last_turns = history[-6:] if history else []
        transcript = ""
        for u, b in last_turns:
            transcript += f"User: {u}\nAssistant: {b}\n"

        full_prompt = (
            f"{system_instruction}\n\n"
            f"{transcript}\n"
            f"User: {message}\nAssistant:"
        )

        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=full_prompt
        )
        return response.text

    except Exception as e:
        return f"⚠️ Error interacting with Gemini: {str(e)}"

Loading weights:   0%|          | 0/200 [00:00<?, ?it/s]

In [ ]:
# ==========================================
# PART 3: MAIN EXECUTION
# ==========================================

# 🔥 Firebase init (set your DB URL)
FIREBASE_DB_URL = ""

print("🔄 Connecting to Firebase Realtime DB...")
try:
    firebase_init(FIREBASE_DB_URL)
    print("✅ Firebase Connected!")
except Exception as e:
    print(f"❌ Firebase Error: {e}")


# --- Load Articles ---
ARTICLE_URLS = [
    "https://www.nature.com/articles/s41467-024-50749-4",
    "https://www.nature.com/articles/s41598-024-72197-2",
    "https://www.nature.com/articles/s41598-022-15163-0",
    "https://www.nature.com/articles/srep16564",
    "https://www.nature.com/articles/s41598-025-15940-7",
]

print("🌐 Loading articles...")
ARTICLES, DOC_TITLES = load_articles_from_web(ARTICLE_URLS)
print(f"✅ Articles loaded: {len(ARTICLES)}")

engine = LectureSearchEngine()


# Try to load from Firebase first, else build
index_loaded = load_index_from_firebase(engine)

engine.documents = ARTICLES
engine.titles = DOC_TITLES

if not index_loaded and ARTICLES:
    # If not in Firebase, BUILD IT from scratch
    print("📚 Building Search Index locally...")
    engine.build_index(ARTICLES, DOC_TITLES)

    # Save newly built index to Firebase
    try:
        msg = save_index_to_firebase(engine)
        print(msg)
    except Exception as e:
        print(f"❌ Failed saving index to Firebase: {e}")
elif index_loaded:
    # Use the loaded index, just ensure it has access to document text for context
    engine.documents = ARTICLES
    engine.titles = DOC_TITLES
    print("⏩ Skipped build. Using Cloud Index.")


# 2. Setup AI Models
ACTIVE_MODEL = get_working_model()
print("🤖 Text AI ready (google.genai client, model set per request)")
print("🧠 Vision model will load only when you click 'Analyze Leaf' in Tab 1.")

# 3. Launch Gradio UI
print("\n🚀 Launching System...")

theme = gr.themes.Soft(
    primary_hue="green",
    secondary_hue="emerald",
).set(
    body_background_fill="#f3f4f6",
    background_fill_primary="#f3f4f6"
)


with gr.Blocks(theme=theme, title="Smart Plant System") as demo:

    # Header Section
    with gr.Row():
        with gr.Column(scale=5):
            gr.Markdown("# 🌱 Smart Plant System Ultimate")
        with gr.Column(scale=1):
            mode_btn = gr.Button("🌗 Light/Dark Mode", variant="secondary")
            mode_btn.click(None, None, None, js="() => document.body.classList.toggle('dark')")

    with gr.Tabs():

        # Tab 1: Image AI
        with gr.TabItem("1. Image (AI) 📸"):
            gr.Markdown("### 🍃 Plant Disease Detection")
            with gr.Row():
                with gr.Column():
                    img_input = gr.Image(height=300, label="Upload Leaf Photo 📷", type="numpy")
                    analyze_btn = gr.Button("🔍 Analyze Leaf", variant="primary")
                with gr.Column():
                    img_out = gr.Textbox(label="AI Diagnosis 🤖", lines=4, placeholder="Waiting for image...")
            analyze_btn.click(analyze_image, inputs=img_input, outputs=img_out)


        # Tab 2: IoT Data & Leaderboard
        with gr.TabItem("2. IoT Data 📊"):
            gr.Markdown("### 📡 Sensor Real-Time Value")

            # 1. IoT Values Row
            with gr.Row():
                v1 = gr.Textbox(label="Current Temperature 🌡️", lines=1)
                v2 = gr.Textbox(label="Current Humidity 💧", lines=1)
                v3 = gr.Textbox(label="Current Soil Moisture 🌿", lines=1)

            # Last Updated Label (Visual Proof)
            lbl_update = gr.Markdown("⏳ Waiting for update...", elem_id="update_lbl")

            gr.Markdown("---")

            # --- LEADERBOARD & WATERING SECTION ---
            with gr.Row():
                # Left Side: Controls
                with gr.Column(scale=1):
                    gr.Markdown("### 🎮 Remote Actions")
                    # Water Button
                    btn_water = gr.Button("🚿 Remote Water (+5 pts) 💧", variant="primary", size="lg")

                    gr.Markdown("### ⚙️ Settings")
                    lim = gr.Number(value=20, label="Data Limit 🔢")
                    btn_json = gr.Button("💾 Export as JSON", variant="secondary")
                    f_out = gr.File(height=50, label="Download JSON", visible=False)

                # Right Side: Leaderboard
                with gr.Column(scale=2):
                    gr.Markdown("### 🏆 Top Gardeners (Leaderboard)")
                    leaderboard_table = gr.Dataframe(
                        value=get_leaderboard_df,
                        headers=["Rank", "Player", "Score"],
                        interactive=False,
                        row_count=6
                    )

            # --- LOGIC ---
            # Timer ticks - updates values + timestamp (PARALLEL FETCH)
            timer = gr.Timer(5)
            # Note: We now output to 4 components (3 values + 1 label)
            timer.tick(fetch_sensor_values, inputs=[lim], outputs=[v1, v2, v3, lbl_update])

            # Export JSON (Uses full limit)
            btn_json.click(export_json_data, inputs=[lim], outputs=[f_out])

            # Watering Logic
            btn_water.click(update_leaderboard, inputs=None, outputs=leaderboard_table)

          # Tab: Big Data Observations
        with gr.TabItem("3. Big Data Observations 📈"):
            gr.Markdown(
                "### 🧠 Big Data Analytics on IoT Streams\n"
                "- Computes **min/max** per parameter (MapReduce requirement) + extra insights.\n"
                "- Saves results to Firebase.\n"
            )

            with gr.Row():
                with gr.Column(scale=1):
                    bt1 = gr.Checkbox(label="Temperature 🌡️", value=True)
                    bt2 = gr.Checkbox(label="Humidity 💧", value=False)
                    bt3 = gr.Checkbox(label="Soil Moisture 🌿", value=False)
                    blim = gr.Number(value=20, label="Analytics Limit 🔢")
                    run_bd_btn = gr.Button("🚀 Run Big Data Analytics", variant="primary")

                    refresh_fb_btn = gr.Button("🔄 Load Latest Analytics from Firebase", variant="secondary")

                with gr.Column(scale=3):
                    bd_table = gr.Dataframe(label="Analytics Table (saved to Firebase)")
                    bd_plot = gr.Plot(label="Trend Plot")
                    bd_log = gr.Textbox(label="Logs / Observations", lines=6)

            run_bd_btn.click(
                run_bigdata_pipeline,
                inputs=[bt1, bt2, bt3, blim],
                outputs=[bd_table, bd_plot, bd_log]
            )

            refresh_fb_btn.click(
                load_latest_analytics_from_firebase,
                inputs=[],
                outputs=[bd_table]
            )


        # Tab 3: Search Docs
        with gr.TabItem("4. Search Docs 🔍"):
            gr.Markdown("### 📚 Knowledge Base Search")
            with gr.Row():
                txt_in = gr.Textbox(label="Ask a Question", placeholder="How does photosynthesis work?...", scale=4)
                search_btn = gr.Button("🔎 Search", variant="primary", scale=1)

            res_out = gr.Textbox(label="AI & Doc Results 💡", lines=12)
            search_btn.click(search_engine_rag, inputs=txt_in, outputs=res_out)

        # Tab 4: Dashboard
        with gr.TabItem("5. Dashboard 🎛️"):
            gr.Markdown("### ⚡ Live Monitor")
            dash_btn = gr.Button("🔄 Sync Live Data", variant="primary")

            with gr.Row():
                b1 = gr.Textbox(label="Temp 🌡️")
                b2 = gr.Textbox(label="Humidity 💧")
                b3 = gr.Textbox(label="Soil 🌿")
                b4 = gr.Textbox(label="Status 🚦")

            with gr.Row():
                dp1 = gr.Plot(label="Temp")
                dp2 = gr.Plot(label="Humidity")
            with gr.Row():
                dp3 = gr.Plot(label="Soil")

            dash_btn.click(refresh_dashboard_real, outputs=[dp1, dp2, dp3, b1, b2, b3, b4])

        # Tab 5: Report (UPDATED: Big Data PDF)
        with gr.TabItem("6. PDF Report 📑"):
            gr.Markdown("### 📄 Generate Professional Analytics Report")
            gr.Markdown("Generates a PDF based on the latest **Firebase Analytics** (Big Data Stats) instead of raw cache.")
            report_btn = gr.Button("🖨️ Generate PDF (Big Data)", variant="primary")
            with gr.Row():
                pdf_file = gr.File(label="Download PDF 📥")
                report_log = gr.Textbox(label="Log 📝", lines=2)
            report_btn.click(generate_pdf_report, outputs=[pdf_file, report_log])

        # NEW TAB: Chatbot
        with gr.TabItem("7. AI Chatbot 💬"):
            gr.Markdown("### 🤖 Chat with Plant System AI")
            gr.Markdown("Ask anything about the system, gardening, or data!")

            # Using standard Chatbot UI components
            chatbot = gr.Chatbot(label="Gemini Assistant", height=400)
            msg = gr.Textbox(label="Your Message", placeholder="Ask me about the plant system...")
            clear = gr.Button("Clear Chat")

            def respond(message, chat_history):
                # Call our Gemini helper
                bot_message = chat_with_gemini(message, chat_history)
                chat_history.append((message, bot_message))
                return "", chat_history

            msg.submit(respond, [msg, chatbot], [msg, chatbot])
            clear.click(lambda: None, None, chatbot, queue=False)


demo.launch(inline=True, height=900, debug=True)

🔄 Connecting to Firebase Realtime DB...
✅ Firebase Connected!
🌐 Loading articles...
✅ Loaded [1] Evolution of Phytophthora infestans on its potato host since the Irish potato famine | Nature Communications
✅ Loaded [2] Advancing plant leaf disease detection integrating machine learning and deep learning | Scientific Reports
✅ Loaded [3] A deep learning based approach for automated plant disease classification using vision transformer | Scientific Reports
✅ Loaded [4] Detection of early blight and late blight diseases on tomato leaves using hyperspectral imaging | Scientific Reports
✅ Loaded [5] Bayesian optimized CNN ensemble for efficient potato blight detection using fuzzy image enhancement | Scientific Reports
✅ Articles loaded: 5
⏳ Checking Firebase for existing search index...
⚠️ Loaded index but it doesn't contain common term 'plant' -> treating as invalid.
📚 Building Search Index locally...
✅ Index saved correctly. terms=2480
🤖 Text AI ready (google.genai client, model set per r

/tmp/ipython-input-4111206921.py:73: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=theme, title="Smart Plant System") as demo:
/tmp/ipython-input-4111206921.py:228: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Gemini Assistant", height=400)
/tmp/ipython-input-4111206921.py:228: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(label="Gemini Assistant", height=400)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://79198948ec777f324c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


✅ No new rows to save for soil (dedup OK)
✅ No new rows to save for humidity (dedup OK)
✅ No new rows to save for temperature (dedup OK)
✅ No new rows to save for humidity (dedup OK)
✅ No new rows to save for soil (dedup OK)
✅ No new rows to save for temperature (dedup OK)
✅ No new rows to save for temperature (dedup OK)
✅ No new rows to save for humidity (dedup OK)
✅ No new rows to save for soil (dedup OK)
✅ No new rows to save for humidity (dedup OK)
✅ No new rows to save for temperature (dedup OK)
✅ No new rows to save for soil (dedup OK)
✅ No new rows to save for temperature (dedup OK)
✅ No new rows to save for humidity (dedup OK)
✅ No new rows to save for soil (dedup OK)
✅ No new rows to save for temperature (dedup OK)
✅ No new rows to save for humidity (dedup OK)
✅ No new rows to save for soil (dedup OK)
✅ No new rows to save for temperature (dedup OK)
✅ No new rows to save for temperature (dedup OK)
✅ No new rows to save for soil (dedup OK)
✅ No new rows to save for humidity (de

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/httptools_impl.py", line 416, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1139, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/error

...

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/httptools_impl.py", line 416, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1139, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/error

Loading weights:   0%|          | 0/200 [00:00<?, ?it/s]

✅ No new rows to save for soil (dedup OK)
✅ No new rows to save for humidity (dedup OK)
✅ No new rows to save for temperature (dedup OK)
✅ No new rows to save for temperature (dedup OK)
✅ No new rows to save for soil (dedup OK)
✅ No new rows to save for humidity (dedup OK)
✅ No new rows to save for temperature (dedup OK)
✅ No new rows to save for humidity (dedup OK)
✅ No new rows to save for soil (dedup OK)
✅ No new rows to save for humidity (dedup OK)
✅ No new rows to save for temperature (dedup OK)
✅ No new rows to save for soil (dedup OK)
✅ No new rows to save for humidity (dedup OK)
✅ No new rows to save for temperature (dedup OK)
✅ No new rows to save for soil (dedup OK)
✅ No new rows to save for soil (dedup OK)
✅ No new rows to save for humidity (dedup OK)
✅ No new rows to save for temperature (dedup OK)
✅ No new rows to save for temperature (dedup OK)
✅ No new rows to save for humidity (dedup OK)
✅ No new rows to save for soil (dedup OK)
✅ No new rows to save for humidity (dedup

/tmp/ipython-input-1356661725.py:272: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(figsize=(8, 3.5))


✅ No new rows to save for temperature (dedup OK)
✅ No new rows to save for humidity (dedup OK)
✅ No new rows to save for soil (dedup OK)
✅ No new rows to save for temperature (dedup OK)
✅ No new rows to save for humidity (dedup OK)
✅ No new rows to save for soil (dedup OK)
✅ No new rows to save for soil (dedup OK)
✅ No new rows to save for temperature (dedup OK)
✅ No new rows to save for humidity (dedup OK)
✅ No new rows to save for humidity (dedup OK)
✅ No new rows to save for soil (dedup OK)
✅ No new rows to save for temperature (dedup OK)
✅ No new rows to save for temperature (dedup OK)
✅ No new rows to save for humidity (dedup OK)
✅ No new rows to save for soil (dedup OK)
✅ No new rows to save for soil (dedup OK)
✅ No new rows to save for temperature (dedup OK)
✅ No new rows to save for humidity (dedup OK)
✅ No new rows to save for temperature (dedup OK)
✅ No new rows to save for humidity (dedup OK)
✅ No new rows to save for soil (dedup OK)
✅ No new rows to save for soil (dedup OK)

In [ ]:
import transformers
print(transformers.__version__)


5.0.0
